In [44]:
import os

from bids import BIDSLayout

from glob import glob

from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function

In [51]:
bids_dir = os.path.join('/fetaldata')
output_dir = os.path.join('/fetaldata','derivatives','mialsrtk')
mask_dir = os.path.join('/fetaldata','derivatives','manual_masks')

subject = '01'

layout = BIDSLayout(bids_dir,derivatives=mask_dir)
print(layout)

#Make sure we start from empty derivatives
#import shutil
#shutil.rmtree(output_dir)

BIDS Layout: .../fetaldata | Subjects: 1 | Sessions: 0 | Runs: 6


In [52]:
import subprocess
def run(self, command, env={}, cwd=os.getcwd()):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.run(command, shell=True,
                               env=merged_env, cwd=cwd, capture_output=True)
    return process

In [53]:
from traits.api import *

import nibabel as nib

from nipype.utils.filemanip import split_filename
from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

from nipype.interfaces.mixins import reporting



In [59]:
%%bash 

mialsrtkOrientImage --help


USAGE: 

   mialsrtkOrientImage  [-O <string>] -o <string> -i <string> [--]
                        [--version] [-h]


Where: 

   -O <string>,  --orientation <string>
     Output orientation: axial (default), sagittal, coronal, or RIP

   -o <string>,  --output <string>
     (required)  Output Image File

   -i <string>,  --input <string>
     (required)  Input Image File

   --,  --ignore_rest
     Ignores the rest of the labeled arguments following this flag.

   --version
     Displays version information and exits.

   -h,  --help
     Displays usage information and exits.


   Reorient an image



In [24]:
%%bash
python /usr/local/bin/mialsrtkHistogramNormalization.py

  File "/usr/local/bin/mialsrtkHistogramNormalization.py", line 48
    print 'min ='+str(landmarks['p1'])
                ^
SyntaxError: invalid syntax


CalledProcessError: Command 'b'python /usr/local/bin/mialsrtkHistogramNormalization.py\n'' returned non-zero exit status 1.

In [55]:
class MialsrtkOrientImageInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    in_file = File(desc='Input image',mandatory=True)
    out_postfix = traits.Str("_reoriented", usedefault=True)
    orientation = traits.Str('axial',desc='Desired orientation (axial by default)')

class MialsrtkOrientImageOutputSpec(TraitedSpec):
    out_file = File(desc='Reoriented image')

class MialsrtkOrientImage(BaseInterface):

    input_spec = MialsrtkOrientImageInputSpec
    output_spec = MialsrtkOrientImageOutputSpec
    
    def _run_interface(self, runtime): 
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        out_file = os.path.join(os.getcwd().replace(self.inputs.bids_dir,'/fetaldata'), ''.join((name, self.inputs.out_postfix, ext)))

        cmd = 'mialsrtkOrientImage -i "{}" -o "{}" -O {}'.format(self.inputs.in_file,out_file,self.inputs.orientation)
        
        try:
            print('... cmd: {}'.format(cmd))
            run(self, cmd, env={}, cwd=os.path.abspath(self.inputs.bids_dir))
        except:
            print('Failed')
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        outputs['out_file'] = os.path.join(os.getcwd().replace(self.inputs.bids_dir,'/fetaldata'), ''.join((name, self.inputs.out_postfix, ext)))
        return outputs
    
class MultipleMialsrtkOrientImageInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    input_images = InputMultiPath(File(desc='files to be reoriented', mandatory = True))
    orientation = traits.Str('axial')
    out_postfix = traits.Str("_reoriented", usedefault=True)
    
class MultipleMialsrtkOrientImageOutputSpec(TraitedSpec):
    output_images = OutputMultiPath(File())

class MultipleMialsrtkOrientImage(BaseInterface):
    input_spec = MultipleMialsrtkOrientImageInputSpec
    output_spec = MultipleMialsrtkOrientImageOutputSpec

    def _run_interface(self, runtime):
        #if len(self.inputs.input_images)>0:
        for input_image in self.inputs.input_images:
            ax = MialsrtkOrientImage(bids_dir = self.inputs.bids_dir, in_file = input_image, out_postfix=self.inputs.out_postfix, orientation = self.inputs.orientation)
            ax.run()
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['output_images'] = glob(os.path.abspath("*.nii.gz"))
        return outputs

In [56]:
#Check if mandatory derivatives dataset_description.json exists in derivatives/mialsrtk.
# If not, it is created before running the workflow, otherwise BIDSDataGrabber is not happy and raises an error. 

mialsrtk_dataset_description_json = os.path.join(output_dir,'dataset_description.json')

print('Check for {}'.format(mialsrtk_dataset_description_json))
if not os.access(mialsrtk_dataset_description_json, os.R_OK):
    import json
    data = {'PipelineDescription':{'Name': 'MIAL Super-Resolution ToolKit', 
                                'Version': 'v2.0.0-beta', 
                                'CodeURL': 'https://github.com/sebastientourbier/mialsuperresolutiontoolkit'
                                  },
            'Name': 'MIAL Super-Resolution ToolKit',
            'BIDSVersion': '1.2.0'
           }
    os.makedirs(output_dir)
    with open(mialsrtk_dataset_description_json, 'w+') as outfile:
        json.dump(data, outfile, indent=4)
    print('File {} was created'.format(mialsrtk_dataset_description_json))
else:
    print('File {} already exists'.format(mialsrtk_dataset_description_json))
    

wf = Workflow(name="image_reorientation",base_dir=os.path.join(output_dir,'sub-{}'.format(subject),'nipype'))

bg = Node(interface=BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.index_derivatives = True
bg.inputs.raise_on_empty = False
bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"])}
imgReorient = Node(interface = MultipleMialsrtkOrientImage(),name='image_reorientation')
imgReorient.inputs.bids_dir = bids_dir
imgReorient.inputs.orientation = 'axial'
                  
wf.connect(bg, "T2ws", imgReorient, "input_images")
#wf.connect(bg, "masks", nlmDenoise, "input_masks")

res = wf.run()

Check for /fetaldata/derivatives/mialsrtk/dataset_description.json
File /fetaldata/derivatives/mialsrtk/dataset_description.json already exists
191023-17:40:00,709 nipype.workflow INFO:
	 Workflow image_reorientation settings: ['check', 'execution', 'logging', 'monitoring']
191023-17:40:00,715 nipype.workflow INFO:
	 Running serially.
191023-17:40:00,717 nipype.workflow INFO:
	 [Node] Setting-up "image_reorientation.bids_grabber" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_reorientation/bids_grabber".
191023-17:40:00,721 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")


/opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/bids/layout/layout.py:690: UserWarning: In pybids 0.9.0, the 'extensions' filter was deprecated in favor of 'extension'. The former will stop working in 0.11.0.
  warnings.warn("In pybids 0.9.0, the 'extensions' filter was "


191023-17:40:02,42 nipype.workflow INFO:
	 [Node] Finished "image_reorientation.bids_grabber".
191023-17:40:02,43 nipype.workflow INFO:
	 [Node] Setting-up "image_reorientation.image_reorientation" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_reorientation/image_reorientation".
191023-17:40:02,47 nipype.workflow INFO:
	 [Node] Running "image_reorientation" ("__main__.MultipleMialsrtkOrientImage")
... cmd: mialsrtkOrientImage -i "/fetaldata/sub-01/anat/sub-01_run-1_T2w.nii.gz" -o "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_reorientation/image_reorientation/sub-01_run-1_T2w_reoriented.nii.gz" -O axial
... cmd: mialsrtkOrientImage -i "/fetaldata/sub-01/anat/sub-01_run-2_T2w.nii.gz" -o "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_reorientation/image_reorientation/sub-01_run-2_T2w_reoriented.nii.gz" -O axial
... cmd: mialsrtkOrientImage -i "/fetaldata/sub-01/anat/sub-01_run-3_T2w.nii.gz" -o "/fetaldata/derivatives/mialsrtk/sub-01/nipype/image_reorientation/image_

___

___